[박현우] 간단한 클러스터링 예제
========================

이 노트북에서는 클러스터링을 하는 방식을 간단하게 다뤄봅니다.

In [1]:
%matplotlib inline
from IPython.display import display, HTML

import matplotlib
krfont = {'family' : 'nanumgothic', 'weight' : 'bold', 'size'   : 10}
matplotlib.rc('font', **krfont)

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [3]:
from multiprocessing import Pool
import multiprocessing

cores = multiprocessing.cpu_count()

In [4]:
chosun_data = pd.read_pickle("chosun_news.df")

1. Corpus 로딩하기
---------------------

In [5]:
from konlpy.tag import Mecab, Hannanum, Twitter
mecab = Mecab()
hannanum = Hannanum()
twitter = Twitter()

In [45]:
import re, hanja
import numba

def tokenizeText(text):
    text = ' '.join(hanja.split_hanja(text))
    text = hanja.translate(text, 'substitution')
    return mecab.morphs(text)

print(tokenizeText('[사설] 美가 증거 내밀자 수사 나선 中 이중 플레이'))

['[', '사설', ']', '미', '가', '증거', '내밀', '자', '수사', '나선', '중', '이중', '플레이']


In [46]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.WARNING)

In [47]:
from gensim import corpora

In [48]:
words = []
# using mecab
with Pool(cores) as p:
    words = list(tqdm(p.imap(tokenizeText, chosun_data['title']), total=len(chosun_data)))
# words = [tokenizeText(text) for text in tqdm(chosun_data['content'])]
dictionary = corpora.Dictionary(words)

2. Doc2Vec를 이용한 문서->벡터 변환
------------------------------

In [49]:
counts = {}
for sentence in tqdm(words):
    for word in sentence:
        counts[word] = counts.get(word, 0) + 1

In [50]:
counts['의']

7791

In [51]:
import gensim
from gensim.models.word2vec import Word2Vec, Text8Corpus
from collections import OrderedDict
import itertools

In [52]:
import random

sentences = [gensim.models.doc2vec.LabeledSentence(
        words=words, tags=[i])
        for i, words in enumerate(words)]
tmp = [(i, sentence) for i, sentence in enumerate(sentences)]
random.shuffle(tmp)
train_set = [x[1] for x in tmp[:len(tmp) * 3 // 10]]

In [53]:
w2v_model = Word2Vec(size=64, workers=cores)
w2v_model.build_vocab(words)

In [54]:
w2v_model.train(words)

1835044

In [55]:
w2v_model.similarity('이명박', '대통령')

0.68442629424775003

* paragraph2vec의 알고리즘에서 word2vec를 쓴다.
* Word2Vec은 전체 뉴스에 대해서 하는게 어떤가. 너무 word/content 갯수가 적지 않은가.
* 전처리 과정이 필요한가?

3. sklearn을 이용한 Agglomerative Clustering
------------------------------------------

In [56]:
def vectorizeWords(ws):
    vsum = np.zeros(64)
    fsum = 0.0
    for w in ws:
        if w in w2v_model.vocab:
            f = counts[w]
            if f == 0:
                print(w)
            fsum += 1 / f
            vsum += w2v_model[w] / f
    if fsum < 0.001:
        return np.zeros(64)
    else:
        return vsum / fsum

In [57]:
vectorizeWords(['사설', '미', '증거', '수사', '나선', '중', '이중', '플레이'])

array([-0.0462473 ,  0.01750928, -0.04175988, -0.09447964,  0.11195441,
       -0.15582331,  0.02205649,  0.12746236, -0.04428291, -0.15164726,
        0.10268498, -0.11308526, -0.05921292,  0.09842206, -0.05529129,
       -0.0151269 , -0.07233634,  0.04110032,  0.03676123,  0.05464866,
        0.16633733,  0.07994697,  0.20016701, -0.04838737, -0.02707192,
        0.07987369,  0.11909217,  0.10016239, -0.01708355,  0.01228863,
       -0.18029985, -0.06363225,  0.0827971 ,  0.13032045, -0.08850109,
        0.06418935, -0.05346495,  0.01459601,  0.19059832,  0.11049549,
       -0.08590515, -0.22287377,  0.09977736,  0.10014536, -0.1881095 ,
       -0.02897716,  0.13450139,  0.01518178,  0.03325992,  0.17482849,
       -0.18535069,  0.03411149,  0.07225417, -0.10198789,  0.00993744,
       -0.07259177, -0.04323257,  0.15358721, -0.02899736,  0.02143741,
        0.06557437,  0.01528938, -0.06396108,  0.10954947])

In [58]:
content_vectors = []
for i, sentence in enumerate(tqdm(sentences)):
    v = vectorizeWords(sentence.words)
    content_vectors.append(v)
content_vectors = np.asarray(content_vectors)

In [59]:
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans, SpectralClustering, MeanShift
from spherecluster import SphericalKMeans

In [60]:
cluster_num = 50
# cluster_model = AgglomerativeClustering(n_clusters=cluster_num, linkage="average", affinity='cosine')
# cluster_model = DBSCAN(eps=.30, min_samples=3, metric='cosine', algorithm='brute')
# cluster_model = KMeans(n_clusters=cluster_num)
cluster_model = SphericalKMeans(n_clusters=cluster_num)
# cluster_model = SpectralClustering(n_clusters=cluster_num, affinity='cosine')
# cluster_model = MeanShift(n_jobs=cores)
cluster_model.fit(content_vectors)

cluster_num = len(set(cluster_model.labels_)) - (1 if -1 in cluster_model.labels_ else 0)
print('# of clusters : %d' % cluster_num)
if -1 in cluster_model.labels_:
    noiseNum = np.sum(cluster_model.labels_ == -1)
    totNum = len(content_vectors)
    print('Noise : %d/%d (%.2f%%)' % (noiseNum, totNum, noiseNum / totNum * 100))

# of clusters : 50


In [61]:
cluster_data = []
for cluster_index in range(cluster_num):
    cluster_data.append(chosun_data[cluster_model.labels_ == cluster_index])

In [62]:
cluster_model.cluster_centers_[0]

array([-0.08885995, -0.02858291, -0.06318207, -0.15403497,  0.18674366,
       -0.14204662, -0.01356459,  0.18621448, -0.05422292, -0.15237655,
        0.17264603, -0.11197643, -0.05810412,  0.12914328, -0.04980899,
        0.00574086, -0.05751289,  0.03186012, -0.02751128,  0.0565769 ,
        0.21821891,  0.09075829,  0.25345403, -0.00063967,  0.01146435,
        0.05274861,  0.21249931,  0.1627496 , -0.05932244,  0.00759457,
       -0.14604796, -0.10461886,  0.0333114 ,  0.18465229, -0.13408277,
        0.0895292 , -0.14225566, -0.02120585,  0.2384859 ,  0.1565302 ,
       -0.07917801, -0.27122549,  0.14312742,  0.1747153 , -0.18191067,
       -0.08814703,  0.16891394,  0.039767  ,  0.03557536,  0.22206328,
       -0.12491282,  0.07363712,  0.08502989, -0.1599461 , -0.0014001 ,
       -0.04598263, -0.05407105,  0.16127243, -0.03758598,  0.08545754,
        0.07001866,  0.03577074, -0.11444906,  0.12352018])

In [63]:
for i in range(cluster_num):
    print('Cluster #%d' % i)
    
    display(list(filter(
                lambda x: counts[x[0]] < 100,
                w2v_model.most_similar(
                    positive=[cluster_model.cluster_centers_[i]],
                    topn=10))))
    display(cluster_data[i][['category', 'title']].sample(10))

Cluster #0


[('심판', 0.9881048202514648),
 ('막말', 0.9871180653572083),
 ('탓', 0.9863877892494202),
 ('설득', 0.9855608940124512),
 ('경고', 0.981814980506897),
 ('뜻', 0.9804854393005371),
 ('좌우', 0.979576051235199),
 ('다른', 0.979257345199585),
 ('용납', 0.9791616201400757)]

,category,title
5150,사설ㆍ칼럼 > 내부칼럼 > 사설,[사설] 그래도 우 수석 감싸는 靑과 친박들 지금 제정신인가
37559,정치,"유인태 ""文선배와 저 포함""… 문희상 ""허허, 거 참"""
8167,정치,"우상호 ""우병우·검찰 개혁 더 이상 지켜만 볼 수 없다"" 박지원 ""朴대통령이 계속 ..."
25930,스포츠ㆍ연예 > 스포츠 > 야구 > 프로야구,"그러니까, 홈런 친 저 선수가 누구냐고요"
19588,사설ㆍ칼럼 > 내부칼럼,[권대열 칼럼] 親朴·非朴 왜 싸우는지 알고나 싸우나
47221,국제 > 미국ㆍ중남미,"""오바마가 일본에 위안부 합의 물밑 압박했다"""
57105,정치,박원순과 손잡으려… 朴공약 끌어안는 文대표
51853,정치 > 국회ㆍ정당,"그간 참았다는 듯… 野주류, 안철수 비판 쏟아내"
25115,정치 > 국회ㆍ정당,"非朴 ""최경환 삭발하라"" 親朴 ""김무성 야반도주"""
14547,사설ㆍ칼럼 > 내부칼럼 > 사설,[사설] '격차 해소' 위해 기득권과 싸우겠다는 安 대표


Cluster #1


[('배출', 0.9953632354736328),
 ('한우', 0.9949995875358582),
 ('트럭', 0.99355149269104),
 ('터진', 0.993334174156189),
 ('오염', 0.9929325580596924),
 ('복원', 0.9928935766220093),
 ('주가', 0.9924120306968689),
 ('기관장', 0.9922081232070923),
 ('사무실', 0.9920132160186768),
 ('홈', 0.9919726848602295)]

...

[('그때', 0.9670799970626831),
 ('꾸', 0.9617404937744141),
 ('바꿔', 0.9494377374649048),
 ('냄새', 0.935985803604126),
 ('달걀', 0.9333729147911072),
 ('맞추', 0.9296841621398926),
 ('웰', 0.9282727837562561),
 ('사다리', 0.9282389879226685),
 ('퍼즐', 0.9268965721130371),
 ('생선', 0.926008939743042)]

,category,title
5671,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [443] 바꿔 부르기
66661,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [221] 제목 외우기
6253,사회 > 리빙포인트,[리빙포인트] 발냄새 없애려면
5422,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [444] 암호로 바꿔 쓰기
43317,사회 > 리빙포인트,[리빙포인트] 오한 들 땐 목뒤에 파스를
33458,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [333] 사다리를 타자
55903,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [255] 칼럼 풀어읽기
13988,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [406] 강세 바꿔 읽기
33207,사회 > 리빙포인트,[리빙포인트] 운동화 찌든 때 쉽게 빼려면
27680,"사회 > 두근두근 뇌 운동, 신문은 의사선생님",[두근두근 뇌 운동] [354] 암호로 바꿔 쓰기


Cluster #49


[('예정', 0.992479681968689),
 ('단', 0.9906438589096069),
 ('최다', 0.9904974102973938),
 ('실적', 0.9899871945381165),
 ('영업', 0.9889582395553589),
 ('첫날', 0.9867038726806641),
 ('질주', 0.9860249757766724),
 ('테임즈', 0.9856252670288086),
 ('갤럭시', 0.985379695892334),
 ('확률', 0.9849423766136169)]

,category,title
54276,사고 > 바로잡습니다,"[바로잡습니다] 30일자 A19면 '60돌 자민당, 역사 검증 본부 출범' 기사 외"
19634,문화 > 전시ㆍ공연 > 전시,15번째 베네치아 비엔날레 건축전 28일 개막
54789,스포츠ㆍ연예 > 스포츠 > 축구 > 해외축구,11경기 연속 골 신기록 바디
54045,스포츠ㆍ연예 > 스포츠 > 농구 > 남자프로농구,"9351점, 김주성 농구 통산득점 3위"
45690,사회 > 법원ㆍ검찰ㆍ경찰,"[땅, 땅… 오늘의 판결] 워터파크 몰카범들 重刑… 징역 3년6개월~4년6개월"
34046,nativeAD > 삼성전자,세계 명견 콘테스트에 삼성 갤럭시S6·기어VR 전시 왜?
63309,사고 > 바로잡습니다,[바로잡습니다] 20일자 A39면 '차세대전투기' 사설 중
46224,스포츠ㆍ연예 > 스포츠 > 종합 > 바둑,"이세돌, 박정환에 명인전 2연승… 바둑왕전 1국도 승리"
3838,정치,1·2黨 대표가 동갑… '58년 개띠' 전성시대
27957,사회 > 날씨,오늘 전국에 황사 섞인 비


In [ ]:
end = if # Intentionally left errornous